In [2]:
!pip install wordcloud arabica awswrangler

  Obtaining dependency information for wordcloud from https://files.pythonhosted.org/packages/90/be/1a7a488f5edcfae6746ffb91e792a1795b6cc058364ea6888b3878d3476f/wordcloud-1.9.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for arabica from https://files.pythonhosted.org/packages/fc/56/a81dda7f5c975eb978486e08cf1542dbc44194d33756be61934aa364187b/arabica-1.7.6-py3-none-any.whl.metadata
  Using cached arabica-1.7.6-py3-none-any.whl.metadata (21 kB)
  Obtaining dependency information for awswrangler from https://files.pythonhosted.org/packages/59/36/8fac34745e7b5847ec1ba6acce062e2966419ace0b41ae1bc9ebbbb89e1a/awswrangler-3.4.2-py3-none-any.whl.metadata
  Using cached awswrangler-3.4.2-py3-none-any.whl.metadata (23 kB)
  Using cached pandas-1.4.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.7 MB)
  Using cached nltk-3.6.2-py3-none-any.whl (1.5 MB)
  Using cached regex-2022.10.31-cp310-cp310-manylinux_2_17_x86_64.many

In [7]:
import pandas as pd
import numpy as np
import os as os
import statsmodels
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.util import ngrams
import awswrangler as wr
from datetime import date

In [5]:
def query(query, username):
    """Retorna df da consulta ao banco de dados"""
    
    from datetime import datetime
    dt = datetime.today().strftime('%Y-%m-%d-%H-%M-%S-%f')
    path = f"s3://will-sdx-ml-platform-sagemaker-transient/{username}/{dt}"

    df = wr.athena.read_sql_query(query, 
    database=None, 
    ctas_approach=False, 
    s3_output=path)

    return df 

#boto3_session = boto3.Session(region_name='sa-east-1')

In [9]:
nr_amostra = 10000
dias_base = -90 #3 meses pra trás
hierarquia = "hierarquia_4"
data_atual = str(date.today())

In [10]:
query_string = f"""
    
    select 
        msg.id_issue
        , msg.dt_envio_mensagem
        , nm_entidade
        , msg.ds_message
        , novas_tags.hierarquia_4
        , novas_tags.fila_de_atendimento
        , i.is_resolvido_bot
        , case 
        when  nm_tag_n3 in (
                    'w3 - nao recebeu token sms acesso'
                    ,'w3 - nao recebeu token email acesso'
                    ,'w3 - nao recebe token 3ds'
                    ,'w3 - problema envio de notificacao indevida'
                    ,'w3 - pedido de exclusao de push email marketing'                   
                    ,'w3 - falha envio comunicacao logistica') then 'grupo1'
        when  nm_tag_n3 in (
                    'w3 - como acessar extrato ou saldo'
                    ,'w3 - como fazer pix com o credito'
                    ,'n3 - open banking'
                    ) then 'grupo2' else '' end as hierarquia_tag
        , case 
            when trim(novas_tags.hierarquia_3) in ('Aquisição', 'Aprovação') then 'Aquisição'
            when trim(novas_tags.hierarquia_3) in ('Diversificação', 'Marketplace','Product Marketing', 'Crédito Pessoal', 'FGTS') then 'Novos produtos ou campanhas'
            when trim(novas_tags.hierarquia_3) in ('Cash In - Pix', 'Cash Out - Pix') then 'Pix'
            when trim(novas_tags.hierarquia_3) in ('Conta', 'Conta Will','Account','Cash In','Cash Out','Cash In - Growth','Cash Out - Monetization') then 'Conta'
            when trim(novas_tags.hierarquia_3) in ('Atendimento', 'Bot') then 'Atendimento'
            when lower(novas_tags.fila_de_atendimento) = 'pagamento' then 'Pagamento'
            when lower(novas_tags.fila_de_atendimento) = 'cobrança' then 'Acordos'
        else novas_tags.hierarquia_3 
        end as hierarquia_analise
    from 
        cx_curated_zone.helpshift_issues_messages msg
    left join cx_curated_zone.helpshift_tag_niveis AS helpshift_tag 
        on msg.id_issue = helpshift_tag.id_issue
    left join processed_zone_api_cxm_tags.tags_hierarquia_gerencial as novas_tags 
        on trim(lower(n3)) = trim(lower(nm_tag_n3))
    left join  cx_curated_zone.helpshift_issues i
     	on i.id_issue = helpshift_tag.id_issue
    where 
        ds_entidade = 'cliente'
        and dt_envio_mensagem >= date_trunc('month', date_add('day', {dias_base},now()))
        and lower(ds_message) <> 'screenshot sent'
        and lower(ds_message) <> 'ok'
        and lower(ds_message) <> 'outros assuntos'
        and lower(ds_message) <> 'não, ainda tenho dúvidas'
        and lower(ds_message) <> 'obrigado'
        and lower(ds_message) <> 'obrigada'
        and lower(ds_message) <> 'quero falar de outra coisa'
        and lower(ds_message) <> 'quero falar com atendimento'
        and lower(ds_message) <> 'quero falar com atendente'
        and lower(ds_message) <> 'nenhuma das opções'
        and lower(ds_message) <> 'certo'
        and lower(ds_message) <> 'responde'
        and lower(ds_message) <> 'falar com atendente'
        and lower(ds_message) <> 'é outra coisa'
        and lower(ds_message) <> 'nenhuma dessas'
        and lower(ds_message) <> 'sim'
        and lower(ds_message) <> 'não'
        and lower(ds_message) <> 'nao'
        and lower(ds_message) <> 'oi'
        and lower(ds_message) <> '?'
        and lower(ds_message) <> '??'
        and lower(ds_message) <> 'bom dia'
        and lower(ds_message) <> 'boa tarde'
        and lower(ds_message) <> 'boa noite'
    order by rand() desc

    """

query_string = query_string + "limit " + str(nr_amostra)

In [11]:
df = query(query_string, 'flavia-costa')

In [12]:
len(df)

10000

In [ ]:
df.columns

In [ ]:
df['is_resolvido_bot'].fillna(0, inplace = True)

In [ ]:
df["ds_message_tratada"] = df["ds_message"].replace(r"[0-9]", " ", regex=True)
df["ds_message_tratada"] = df["ds_message_tratada"].replace("\t", " ", regex=False)
df["ds_message_tratada"] = df["ds_message_tratada"].replace("\n", " ", regex=False)
df["ds_message_tratada"] = df["ds_message_tratada"].replace(" ", "", regex=False)
    
df_clean = (
        df.query('ds_message_tratada != " "')
        .query('ds_message_tratada != "  "')
        .query('ds_message_tratada != ""')
        .query('ds_message_tratada != "   "')
        .query(f"{hierarquia}.notnull()")
    )

In [ ]:
################## df clean para resolvidos pelo Bot sim ou não ########################
df_clean_bot = df_clean.query('is_resolvido_bot == 1')
df_clean_transbordo = df_clean.query('is_resolvido_bot == 0')

In [ ]:
print("Initiating Loop Bot")
df_trigram_bot = pd.DataFrame()

    for i in df_clean_bot[hierarquia].unique():        
        try:
            df_assunto = df_clean_bot[df_clean_bot[hierarquia] == i]           
            df_agg_tmp = arabica_freq(
                text=df_assunto["ds_message_tratada"],
                time=df_assunto["dt_envio_mensagem"],
                time_freq="W",
                max_words=3,
                stopwords=["portuguese"],
                skip=[stopwords],
                numbers=True,
                punct=True,
                lower_case=True,
            )
            df_agg_tmp["termo1"] = df_agg_tmp["trigram"].str.split(":").str[0]
            df_agg_tmp["termo2"] = df_agg_tmp["trigram"].str.split(":").str[1]
            df_agg_tmp["termo3"] = df_agg_tmp["trigram"].str.split(":").str[2]
            df_agg_tmp["freq_termo3"] = df_agg_tmp["trigram"].str.split(":").str[3]
            regex = r"(\d+),"
            df_agg_tmp["freq_termo1"] = (
                df_agg_tmp["termo2"]
                .apply(lambda x: re.sub(regex, r"\1-", x)).astype("str")
                .str.split("-")
                .str[0]
            )
            df_agg_tmp["termo2"] = (
                df_agg_tmp["termo2"]
                .apply(lambda x: re.sub(regex, r"\1-", x)).astype("str")
                .str.split("-")
                .str[1]
            )
            df_agg_tmp["freq_termo2"] = (
                df_agg_tmp["termo3"]
                .apply(lambda x: re.sub(regex, r"\1-", x)).astype("str")
                .str.split("-")
                .str[0]
            )
            df_agg_tmp["termo3"] = (
                df_agg_tmp["termo3"]
                .apply(lambda x: re.sub(regex, r"\1-", x)).astype("str")
                .str.split("-")
                .str[1]
            )
            df_agg_tmp_m = df_agg_tmp.melt(
                id_vars=["period"],
                value_vars=["termo1", "termo2", "termo3"],
                var_name="ranking",
                value_name="descricao",
            ).join(
                df_agg_tmp.melt(
                    id_vars=["period"],
                    value_vars=[
                        "freq_termo1",
                        "freq_termo2",
                        "freq_termo3"
                    ],
                    var_name="freq",
                    value_name="frequencia",
                )[["frequencia"]]
            )
            df_agg_tmp_m["period"] = df_agg_tmp_m["period"].astype("string")
            df_agg_tmp_m["frequencia"] = df_agg_tmp_m["frequencia"].astype("int")
            df_agg_tmp_m = df_agg_tmp_m[['period','descricao', 'frequencia']]
            df_agg_tmp_m = df_agg_tmp_m.sort_values(by = ['period', 'frequencia'], ascending = False)
            df_agg_tmp_m['Tema'] = i
            df_agg_tmp_m = df_agg_tmp_m
            j = i.replace(" ", "")
            j = j.replace("/", "_")
            df_agg_tmp_m['rank'] = df_agg_tmp_m['frequencia'].rank(na_option='bottom', ascending=False)
            df_trigram_bot = df_trigram_bot.append(df_agg_tmp_m)

        except (IndexError, ValueError, TypeError):
            pass
    
    agg_bot = df_trigram_bot[['descricao', 'period']].drop_duplicates().groupby('descricao').agg({'period':['count']})
    agg_bot.columns = agg_bot.columns.droplevel(0)
    agg_bot = agg_bot.reset_index()
    df_trigram_bot = df_trigram_bot.join(agg_bot.set_index('descricao'), on = 'descricao', how = 'left' )
    df_trigram_bot = df_trigram_bot.rename(columns={'count':'count_dias', 'descricao':'trigram', 'period':'dt_week_issue', 'Tema':hierarquia})
    df_trigram_bot['dt_processing'] = data_atual 
    df_trigram_bot['is_resolvido_bot'] = 1

############################## LOOP TRANSBORDO #########################################################
    print("Initiating Loop Transbordo")
    df_trigram_transbordo = pd.DataFrame()

    for i in df_clean_transbordo[hierarquia].unique():        
        try:
            df_assunto = df_clean_transbordo[df_clean_transbordo[hierarquia] == i]           
            df_agg_tmp = arabica_freq(
                text=df_assunto["ds_message_tratada"],
                time=df_assunto["dt_envio_mensagem"],
                time_freq="W",
                max_words=3,
                stopwords=["portuguese"],
                skip=[stopwords],
                numbers=True,
                punct=True,
                lower_case=True,
            )
            df_agg_tmp["termo1"] = df_agg_tmp["trigram"].str.split(":").str[0]
            df_agg_tmp["termo2"] = df_agg_tmp["trigram"].str.split(":").str[1]
            df_agg_tmp["termo3"] = df_agg_tmp["trigram"].str.split(":").str[2]
            df_agg_tmp["freq_termo3"] = df_agg_tmp["trigram"].str.split(":").str[3]
            regex = r"(\d+),"
            df_agg_tmp["freq_termo1"] = (
                df_agg_tmp["termo2"]
                .apply(lambda x: re.sub(regex, r"\1-", x)).astype("str")
                .str.split("-")
                .str[0]
            )
            df_agg_tmp["termo2"] = (
                df_agg_tmp["termo2"]
                .apply(lambda x: re.sub(regex, r"\1-", x)).astype("str")
                .str.split("-")
                .str[1]
            )
            df_agg_tmp["freq_termo2"] = (
                df_agg_tmp["termo3"]
                .apply(lambda x: re.sub(regex, r"\1-", x)).astype("str")
                .str.split("-")
                .str[0]
            )
            df_agg_tmp["termo3"] = (
                df_agg_tmp["termo3"]
                .apply(lambda x: re.sub(regex, r"\1-", x)).astype("str")
                .str.split("-")
                .str[1]
            )
            df_agg_tmp_m = df_agg_tmp.melt(
                id_vars=["period"],
                value_vars=["termo1", "termo2", "termo3"],
                var_name="ranking",
                value_name="descricao",
            ).join(
                df_agg_tmp.melt(
                    id_vars=["period"],
                    value_vars=[
                        "freq_termo1",
                        "freq_termo2",
                        "freq_termo3"
                    ],
                    var_name="freq",
                    value_name="frequencia",
                )[["frequencia"]]
            )
            df_agg_tmp_m["period"] = df_agg_tmp_m["period"].astype("string")
            df_agg_tmp_m["frequencia"] = df_agg_tmp_m["frequencia"].astype("int")
            df_agg_tmp_m = df_agg_tmp_m[['period','descricao', 'frequencia']]
            df_agg_tmp_m = df_agg_tmp_m.sort_values(by = ['period', 'frequencia'], ascending = False)
            df_agg_tmp_m['Tema'] = i
            df_agg_tmp_m = df_agg_tmp_m
            j = i.replace(" ", "")
            j = j.replace("/", "_")
            df_agg_tmp_m['rank'] = df_agg_tmp_m['frequencia'].rank(na_option='bottom', ascending=False)
            df_trigram_transbordo = df_trigram_transbordo.append(df_agg_tmp_m)

        except (IndexError, ValueError, TypeError):
            pass
    
    agg_transbordo = df_trigram_transbordo[['descricao', 'period']].drop_duplicates().groupby('descricao').agg({'period':['count']})
    agg_transbordo.columns = agg_transbordo.columns.droplevel(0)
    agg_transbordo = agg_transbordo.reset_index()
    df_trigram_transbordo = df_trigram_transbordo.join(agg_transbordo.set_index('descricao'), on = 'descricao', how = 'left' )
    df_trigram_transbordo = df_trigram_transbordo.rename(columns={'count':'count_dias', 'descricao':'trigram', 'period':'dt_week_issue', 'Tema':hierarquia})
    df_trigram_transbordo['dt_processing'] = data_atual 
    df_trigram_transbordo['is_resolvido_bot'] = 0